In [15]:
#!/usr/bin/env python
# coding: utf-8

# In[11]:


import numpy as np
import sys
import nbconvert
from metpy.calc import (dewpoint_from_specific_humidity,
                        dewpoint_from_relative_humidity,
                        mixing_ratio_from_relative_humidity, 
                        relative_humidity_from_dewpoint,
                        find_intersections,
                        el,
                        lfc,
                        parcel_profile,
                        cape_cin,
                        lcl,
                        equivalent_potential_temperature,
                        specific_humidity_from_mixing_ratio,
                        relative_humidity_from_mixing_ratio,
                        mixing_ratio_from_relative_humidity)
from metpy.units import units
import datetime as dt
import matplotlib.pyplot as plt
#from read_func import read_AIRS_L3
#from plot_func import draw_skewT
## parameters
g           = 9.81;
p00         = 100000;
cpd         = 1005.7;
Rd          = 287.04;
Rv          = 461.5;
t0          = 273.15;
cpv         = 1875;
cpl         = 4190;
cpi         = 2118.636;
reps        = Rv / Rd;
rddcp       = Rd / cpd;
cpdg        = cpd/g;
converge    = 0.0001;
xlv         = 2501000;
xls         = 2836017;
lv1         = xlv + (cpl - cpv) * t0;
lv2         = cpl - cpv;
ls1         = xls + (cpi - cpv) * t0;
ls2         = cpi - cpv;  
#==============================================================================
def get_qvs(p,t):
    #t is kelvin degree
    #function to calculate saturated mixing ratio
    #if input is Td instead of T, output is qv    
    es = 611.2 * np.exp(17.67 * (t - 273.15)/(t - 29.65))
    qvs = 287.04/461.5 * es / (p - es) #混合比 大气物理P20,2.2.4
    return qvs
#def get_qvs(td,t,p):
#    #t is kelvin degree
#    #function to calculate saturated mixing ratio
#    #if input is Td instead of T, output is qv 
#    qvs = mixing_ratio_from_relative_humidity(
#            relative_humidity_from_dewpoint(t*units('K'),td*units('K')),
#                            t*units('K'),(p/100)* units('hPa'))
##    es = 611.2 * np.exp(17.67 * (t - 273.15)/(t - 29.65))
##    qvs = 287.04/461.5 * es / (p - es) #混合比 大气物理P20,2.2.4
    return qvs
def get_qvi(p,t):
    #function to calculate saturated water vapor mixing ratio for ice
    #Magnus,1967
    #Bolton 1980, t< 0,大气物理P21-2.2.11, 
    es  = 610.78 * np.exp(21.8745584 * (t - 276.16) / (t - 7.66))
    qvi = 287.04/461.5 * es / (p - es)
    return qvi

def get_entropy(p,t,rv,rl,ri):
    #function to calculate entropy
    #Hauf and Holler 1987 (3.13)
    s0d = 6775; 
    #s0v = 10320; 
    s0l = 3517; 
    #s0i = 2296;
    xlv = 2501000
    xls = 2836017
    lv1 = xlv + (cpl - cpv) * t0
    lv2 = cpl - cpv
    ls1 = xls + (cpi - cpv) * t0
    ls2 = cpi - cpv
#    lv0 = 2501000
#    ls0 = 2836017
   
    rt = rv + rl + ri;  # total water mixing ratio
    qd = 1 / (1 + rt);  #dry air percentage (not mixing ratio)
    qt = rt * qd;       #total water specific humidity
    qv = rv * qd;       #vapor specific humidity
    qi = ri * qd;       #ice specific humidity
    # ql = rl * qd;     # liquid specific humidity
    pv = Rv*rv / (Rd + Rv*rv) * p   # partial vapor pressure of parcel
    pd = p - pv                     # partial pressure of dry air
    esl = 611.2  * np.exp(17.67 * (t - t0) / (t - 29.65));
    esi = 610.78 * np.exp(21.8745584 * (t - t0) / (t - 7.66));
    Llv = lv1 - lv2 * t;
    Liv = ls1 - ls2 * t;
#    Llv = lv0 - (cpl - cpv) * (t - t0 )
#    Liv = ls0 - (cpi - cpv) * (t - t0 )
    Lil = Liv - Llv
    Alv = Rv*t * np.log(esl/pv )
    Ail = Rv*t * np.log(esi/esl)
   # print(t/t0)
    #print(pd,p00,pd/p00)
    sd = cpd * np.log(t/t0) - Rd * np.log(pd/p00) + s0d;
    #sv = cpv * log(t/t0) - Rv * log(pv/p00) + s0v;
    sl = cpl * np.log(t/t0) + s0l;
    #si = cpi * log(t/t0) + s0i;
    
    s = sd * qd + sl * qt + ((Llv+Alv)*qv - (Lil+Ail)*qi)/ t;
#    print('S:',s)
    #s = sd * qd + sv * qt + (- (Llv+Alv)*ql - (Liv+Aiv)*qi) / t;
    #s = sd * qd + sv * qv + sl * ql + si * qi;
    return s
source_list={0:['Start from specified height level'],
             1:['Start from surface'],
             2:['Start from specified pressure level'],}
            
h_input_list={1:['dewpoint'],
             2:['mixing ratio'],
             3:['relative humidity'],}

entrainment_opt_list={0:['No Entrainment'],
                      1:['Constant entrainment rate'],
                      2:['DIA scheme'],}


# In[13]:


#=============================================================================
class cal_buoyancy():                   
    """   
===============================================================================
   Yizhou Zhuang
   Department of Atmospheric and Oceanic Sciences,
   School of Physics, Peking University
   zyz90@pku.edu.cn
   Last updated: 05/11/2017
===============================================================================
   Part of this function is ported from a Fortran subroutine written by
   George H. Bryan, which is used in CM-1 model

   References:  Bolton (1980, MWR)             (constants and definitions)
                Bryan and Fritsch (2004, MWR)  (ice processes)
                Hauf and Holler (1987, JAS)    (entropy)
                Holloway and Neelin (2009, JAS)(DIA/DIB entrainment)
===============================================================================
   Input: 
       t_profile,         # temperature profile
       h_profile,         # humidigy profile
       p_profile,         # units:hPa    
       alt_profile,         # alt  unit:m

   Output: 
           bp           Buoyancy                                     (m/s2)
           tp           parcel temperature                          (degK)
           qp           parcel mixing ratio of water vapor, liquid and ice(g/kg)
   User option within the code, defult set:
          td_r_q              input option for humidity profile,1:dewpoint,2:mixing ratio,3:relative humidity, other:specific humidity（units:g/kg）
          adiabat             Formulation of moist adiabat,1: all condensate fall out,2:max mixing ratio specified
          yaxis_base          yaxis base, 0: altitude y axis, 1: pressure y axis
          source              decie where to lift, 0 from surface, 1 from mixd layer
          prc_w               percentage per m (0-1) of condensate kept in the parcel
          qc_max              max mixing ratio of condensate in cloud
          ice                include ice process
          tice               temperature at which all condensate will be ice form 
          newton_step_dt     newton method step distence
          switch_off_ent_t     whether get rid of temperature entrainment 
          switch_off_ent_r     whether get rid of humidity  entrainment
   User option within the code:
           Z         - height of the profile (calculate if not given)  (m)
           SOURCE    - source of parcel
                       0 = specific level (specified by zs)
                       1 = surface (first level with non-nan value)
           ZS        - parcel origin level (source=0)                  (m)
                       default: surface
           ICE       - ice process option
                       0 = no ice process, liquid only
                       1 = Tao et al 1989
           TICE      - temperature at which condensate all freeze (ice=1)
                       default: -40 degC                            (degC)
           ADIABAT   - formulation of moist adiabat:
                       0 = condensate all kept in parcel
                       1 = condensate all fall out of parcel
                       2 = max condensate specified
                       3 = condensate fall out with a certain ratio
           QW_MAX    - max condensate mixing ratio in cloud (adiabat=2)
                       default: 1                                   (g/kg)
           PRC_W     - percentage of condensate falling out of parcel
                       (adiabat=3) default: 0.5/km                  (km-1)
           ENTRAIN_OPT - entrainment option
                       0 = no entrainment
                       1 = constant entrainment
                       2 = DIA entrainment
                       3 = DIB entrainment (not implemented yet)
           X_EN      - fractional entrainment rate (entrain_opt=1)  (km-1)
                       default: 0.1/km
           CE        - constant related to entrainment (entrain_opt=2)
           """
    def __init__(self,
                t_profile,         # temperature profile
                h_profile,         # humidigy profile-units:g/kg
                p_profile,         # units:hPa
                alt_profile,       #
                td_r_q   = 1  ,    # input option for humidity profile
                plot_index = 1,
                source   = 1,      # source of parcel
                z_opt    = 0,      # height is calculated, default(0) not calculated
                z0       = 0,      # surface hegith
                zs       = 0,      # parcel start height/presure,depends on z_opt
                adiabat  = 1,      # Formulation of moist adiabat
                entrain_opt = 1,   # entrainment option
                ce       = 0.4 ,    # parameter related to calculation of entrainment rate
                x_en     = 0.1/1000,   # constant entrainment rate
                prc_w    = 0.0005 , # percentage per m (0-1) of condensate kept in the parcel
                qc_max   = 0.001,   # max mixing ratio of condensate in cloud
                ice      = 1  ,   # include ice process
                tice     = 233.15 , # temperature at which all condensate will be ice form
                tp_te    = 0  ,    # option to include the coefficient tp/te in some calculation
                
                newton_step_dt   = 10,# newton method step distence
                switch_off_ent_t = 0,#whether get rid of temperature entrainment 
                switch_off_ent_r = 0, #whether get rid of humidity  entrainment
                ):
       
        #print(self.__doc__)
        self.t_in = t_profile    
        self.h_in = h_profile
        self.p_in = p_profile*100   # tansfer unit to Pa
        self.z_in = alt_profile
        self.ind  = ~np.isnan(self.z_in)
        self.t    = self.t_in[self.ind]
        if np.nanmean(self.t) < 100:
            self.t_in = t_profile+t0
            self.t = t0 + self.t       # covert from Celsius To Kelvin temperature
        self.h    = self.h_in[self.ind]
        self.p    = self.p_in[self.ind]
        self.z    = self.z_in[self.ind]
        self.td_r_q           = td_r_q  
        self.source           =source
        self.z_opt            =z_opt  
        self.z0               =z0       
        self.adiabat          =adiabat  
        self.entrain_opt      =entrain_opt 
        self.ce               =ce      
        self.x_en             =x_en     
        self.prc_w            =prc_w    
        self.qc_max           =qc_max   
        self.ice              =ice      
        self.tice             =tice     
        self.tp_te            =tp_te             
        self.switch_off_ent_t =switch_off_ent_t 
        self.switch_off_ent_r =switch_off_ent_r 
        self.newton_step_dt   =newton_step_dt
        
        self.read_index=1 
        self.data_process()
        self.process() 
        #self.info()
        if plot_index:
            self.plot_t()

    def info(self):      
        print('\tENTRAIN SCHEME:',entrainment_opt_list[self.entrain_opt][0])
        print('\tMOSITURE INPUT:',h_input_list[self.td_r_q][0])
        print('\tSOURCE:',source_list[self.source][0])
        print('\tPressure(Pa):',self.p[0])
        print('\tAltitude(m):',self.z[0])
        print('\tTemperature(K):',self.t[0])
        print('\tDewpoint(K):',self.td[0])
        print('\tMixingRatio(g/g):',self.q[0])
        print()
#==============================================================================
#==============================================================================
    #initialize output variables
    def data_process(self):
        
        #input is dewpoint
        if self.td_r_q == 1: 

            if np.nanmean(self.h) < 100:

                self.td = t0 + self.h       # covert from Celsius To Kelvin temperature
            else:
                self.td = self.h
                print("hello??")
            self.q = get_qvs(self.p,self.td)

#            self.q = mixing_ratio_from_relative_humidity(relative_humidity_from_dewpoint(self.t*units('K'),self.td*units('K')),
#                                                         self.t*units('K'),(self.p/100)* units('hPa'))
#        #input is mixing ratio
        elif self.td_r_q == 2:
            
            self.q  = self.h/1000   #单位换算为g/g
            self.td = np.array(dewpoint_from_specific_humidity(specific_humidity=(self.q/(1+self.q)), 
                                                      temperature=self.t*units('K'), 
                                                      pressure=(self.p/100)* units('hPa')).to(units('K')))
            #plt.plot(self.p,self.q)
            #self.plot_t()                     
        #input is relative humidity
        elif self.td_r_q ==3:
            self.q = mixing_ratio_from_relative_humidity(relative_humidity=self.h,
                                                       temperature=self.t*units('K'),
                                                       pressure=(self.p/100)* units('hPa'))
            self.td = np.array(dewpoint_from_relative_humidity(self.t*units('K'),self.h).to(units('K')))
            #self.plot_t() 
            #plt.plot(self.p,self.q)
        #input is specific humidity
        else: 

            q = self.h/ 1000 #单位换算为g/g
            self.q = q / (1-q) 
            self.td = np.array(dewpoint_from_specific_humidity(self.q,self.t,self.p))
        
        self.prof = parcel_profile(self.p*units('Pa'), self.t[0]*units('K'), self.td[0]*units('K')).to('degC')
        pi  = np.power((self.p/p00),rddcp)              #ratio between temperature and potential temperature
        self.tv  = self.t* (1+ reps*self.q)/(1+self.q)  #virtual temperature
        self.thv = self.tv / pi                         #virtual potential temperature
        # s   = get_entropy(p, t, q, np.zeros(len(q)), np.zeros(len(q)));   # entropy,why?
        #==============================================================================
        #find source parcel
        if self.z_opt==1: #未给定高度，从0开始抬升
            self.z0  =  0
            self.dzc = -cpdg*0.5*(self.thv[1:]+self.thv[0:-1])*(pi[1:]-pi[0:-1]) 
            self.z   = np.cumsum(np.append(self.z0,self.dzc)) 
        if  self.source == 1:     #surface parcel           
            self.kmax = 0
        elif self.source == 0:   #specified start height level
        #    kmax=find(z>=zs,1,'first') in matlab
            self.kmax = np.where(self.z >= self.zs)[0][0]
        elif self.source == 2:   #specified start pressure level
        #    kmax=find(z>=zs,1,'first') in matlab
            self.kmax = np.where(self.p >= self.zs)[0][0]
        else:
            sys.stderr.write("Unknown value for source")
            raise SystemExit(1)
        self.init_output_vari()
#        self.process()
        #print(len(self.bp[self.bp>0]))
#        if len(self.bp[self.bp>0])>=5:
#           # print(self.bp)
#            #self.plot_t()
#            self.index = 1
#            return       
    def init_output_vari(self):
        self.bp = np.full([len(self.p_in), 1] , np.nan)
        self.tp = np.full([len(self.p_in), 1] , np.nan)
        self.qp = np.full([len(self.p_in), 3] , np.nan)
        self.other_zc     = np.full([len(self.p_in),  1], np.nan)
        self.other_dbpdzs = np.full([len(self.p_in),  9], np.nan)
        self.other_dbpdze = np.full([len(self.p_in), 10], np.nan)
        self.other_dbpdzp = np.full([len(self.p_in),  2], np.nan)
        self.X = np.full((len(self.p_in),),np.nan)
        
        self.td_in = np.full([len(self.p_in), 1] , np.nan)
        #initialize another group of temperary variables
        self.bp0     = np.full([len(self.p),  1], np.nan)
        self.tp0     = np.full([len(self.p),  1], np.nan)
        self.qp0     = np.full([len(self.p),  3], np.nan)
        self.dbpdzs0 = np.full([len(self.p),  9], np.nan)
        self.dbpdze0 = np.full([len(self.p), 10], np.nan)
        self.dbpdzp0 = np.full([len(self.p),  2], np.nan)
    
    def process(self):
        #process contains ascent of parcel, isentropic process, entrainment process, precipitation process
        #==============================================================================
        #define parcel properties at initial location    
        k   = self.kmax
        p2  = self.p[self.kmax]  # pressure
        t2  = self.t[self.kmax]  # temperature
        qv2 = self.q[self.kmax]  # mixing ratio
        ql2 = 0        # liquid mixing ratio
        qi2 = 0        # ice mixing ratio
        qt  = qv2
        b2  = 0        # buoyancy
        # m2 = 1
        #print(p2,t2,qv2)
        
        doit = True
        #==============================================================================
        #begin ascent of parcel
        nk       = len(self.p) 
        self.bp0[k]   = b2
        self.tp0[k]   = t2
        self.qp0[k,:] = [qv2,0,0]
        X_list = np.full((nk,),np.nan)
        while doit and k < nk-1 :
            k = k+1
            #print(k)
            if self.z_opt == 1 :
                dz = self.dzc[k-1]
            else :
                dz = self.z[k] - self.z[k-1]
            #values from last step
            p1  = p2
            p2  = self.p[k]
            t1  = t2
            qv1 = qv2
            ql1 = ql2
            qi1 = qi2
        #    print(p1,p2,t1,t2)
            s1  = get_entropy(p1, t1, qv1, ql1, qi1)
        #    print('s1:',s1)
        #==============================================================================   
        # Isentropic process 
        #get_entropy --> fs1/fs2 --> dt2 --> t2 --> bp
        # 牛顿法逼近
            i  = 0;
            not_converged = True
            while not_converged:  
                i  = i + 1
                if self.ice == 1 :
                    #proportion of liquid to ice
                    fliq = np.max([np.min([(t2-self.newton_step_dt-self.tice)/(t0-self.tice), 1]), 0]);
                    fice = 1 - fliq
                else:
                    fliq = 1
                    fice = 0
                #保证湿度不凭空增加以及冰相湿度不为负，两个公式在数值较小的情况下也会有此情况发生
                #"Ensure that humidity does not increase out of thin air, 
                #and the ice-phase humidity does not become negative; 
                #these two equations may also exhibit this behavior in cases of small numerical values."
                
                #qv2 = mixing ratio
                #qi2 = ice mixing ratio
                #ql2 = liquid water mixing ratio
                
                qv2 = np.min([qt, fliq * get_qvs(p2, t2-self.newton_step_dt) + fice * get_qvi(p2, t2 - self.newton_step_dt)])
                qi2 = np.max([fice * (qt - qv2), 0])
                ql2 = np.max([qt - qv2 - qi2, 0])
                #print(p2, t2-newton_step_dt, qv2, ql2, qi2)
                fs2 = get_entropy(p2, t2-self.newton_step_dt, qv2, ql2, qi2) - s1
                
                if self.ice == 1 :
                    fliq = np.max([np.min([(t2-self.tice)/(t0-self.tice), 1]), 0]);
                    fice = 1 - fliq;
                else :
                    fliq = 1;
                    fice = 0;
                
                qv2 = np.min([qt, fliq * get_qvs(p2, t2) + fice * get_qvi(p2, t2)])
                qi2 = np.max([fice * (qt - qv2), 0])
                ql2 = np.max([qt - qv2 - qi2, 0])
                ##print(p2, t2, qv2, ql2, qi2)
                fs1 = get_entropy(p2, t2, qv2, ql2, qi2) - s1
                
                dt2 = self.newton_step_dt * fs1 / (fs2 - fs1) 
        #        print("k,i:",k,i)
        #        print("dt2:",dt2)
                
                #牛顿法迭代90次以上不收敛考虑调整参数       
                if i > 90:
                    print([k, self.z[k], t2, dt2])
                
                if i > 100:
                    print('lack of convergence, stopping iteration.')
                    break  ##需要确认
    
                if np.abs(dt2) > converge: 
                    t2  = t2 + dt2
                else:
                    ##收敛，气块温度计算结束，输出变量
                    
                    not_converged = False;
                    tv2 = t2 * (1 + reps * qv2) / (1 + qt)    #virtual temperature

                    b2  = g * (tv2 - self.tv[k])/self.tv[k]
                    #store parcel temperature gradient related to different process
                    tbar  = 0.5 * (t1 + t2);
                    qvbar = 0.5 * (qv1 + qv2);
                    qlbar = 0.5 * (ql1 + ql2);
                    qibar = 0.5 * (qi1 + qi2);
                    lhv   = lv1 - lv2 * tbar;
                    lhs   = ls1 - ls2 * tbar;
                    Rm    = Rd + Rv * qvbar
                    #total specific heat at constant pressure
                    cpm   = cpd + cpv * qvbar + cpl * qlbar + cpi * qibar;
                    tebar = 0.5 * (self.t[k]+self.t[k-1])
    
                    if self.tp_te:
                        self.dbpdzs0[k,0] = - g * tbar/tebar * np.log(self.t[k]/self.t[k-1])       # env
                        self.dbpdzs0[k,1] =   g * tbar/tebar * Rd/cpd * np.log(p2/p1)    # adiabatic
                        self.dbpdzs0[k,2] =   g * tbar/tebar * (Rm/cpm - Rd/cpd) * np.log(p2/p1)  # heat storage by water
                    else :
                        self.dbpdzs0[k,0] = - g * np.log(self.t[k]/self.t[k-1])    # env
                        self.dbpdzs0[k,1] =   g * Rd/cpd * np.log(p2/p1)  # adiabatic
                        self.dbpdzs0[k,2] =   g * (Rm/cpm - Rd/cpd) * np.log(p2/p1)  # heat storage by water
                    
                    self.dbpdzs0[k,3] = - g / tebar * lhv * (qv2 - qv1) / cpm       #condensation
                    self.dbpdzs0[k,4] =   g / tebar * (lhs-lhv) * (qi2 - qi1) / cpm #fusion
                    self.dbpdzs0[k,5] =   g * (reps - 1) * (qv2 - qv1)
                    self.dbpdzs0[k,6] = - g * (reps - 1) * (self.q[k] - self.q[k-1])
                    self.dbpdzs0[k,7] = - g * (ql2 - ql1)
                    self.dbpdzs0[k,8] = - g * (qi2 - qi1)
                    self.dbpdzs0[k,:] = self.dbpdzs0[k,:] / dz
        #        print('t1,t2:',t1,t2)
        #==============================================================================   
        ## Entrainment process (no detrainment)
        ## what is switch_off_ent_t???
            if (self.entrain_opt > 0 ) and ~(self.switch_off_ent_t & self.switch_off_ent_r):
                # backup values before entrainment
                t2_  = t2
                qv2_ = qv2
                ql2_ = ql2
                qi2_ = qi2
                qt_  = qt
                cpm_ = cpd + cpv*qv2_ + cpl*ql2_ + cpi*qi2_;
                #fractional entrainment rate (entrain_opt=1)  (km-1) default: 0.1/km
                #constant related to entrainment (entrain_opt=2)      
                if self.entrain_opt == 1:
                    X = self.x_en * dz;
                    #print(X)
                elif self.entrain_opt == 2:     #DIA scheme
                    X = self.ce * dz / (self.z[k]-self.z0);
                    #print(X)
                else:
                    print('invalid entrainment option.')
                    break
                X_list[k]=X
                # mix parcel and environment
                
                #A_k+1 from paper 
                if self.switch_off_ent_r:
                    qt   = (  qt/(1+qt_) + X * qv2_/(1+qv2_)) / (1+X)
                    qv2m = (qv2_/(1+qt_) + X * qv2_/(1+qv2_)) / (1+X)
                else:
                    qt   = ( qt/(1+qt_) + X * self.q[k]/(1+self.q[k])) / (1+X)
                    qv2m = (qv2/(1+qt_) + X * self.q[k]/(1+self.q[k])) / (1+X)
                
                ql2m = ql2/(1+qt_) / (1+X);
                qi2m = qi2/(1+qt_) / (1+X);
                qv2m = qv2m / (1-qt);  # specific humidity --> mixing ratio
                ql2m = ql2m / (1-qt);   #qt is mixing ratio!!!!
                qi2m = qi2m / (1-qt);
                qt   = qt / (1-qt);
                        
                if self.switch_off_ent_t:
                    cpme = cpd + cpv*self.q[k]
                    cpmm = cpd + cpv*qv2m + cpl*ql2m + cpi*qi2m;
                    t2m  = (1/(1+qt_)*cpm_*t2_ + 1/(1+self.q[k])*cpme*t2_*X) / cpmm*(1+qt) / (1+X);
                    t2md = t2_;
                elif self.switch_off_ent_r:
                    cpme = cpd + cpv*qv2m;
                    cpmm = cpd + cpv*qv2m + cpl*ql2m + cpi*qi2m;
                    t2m  = (1/(1+qt_)*cpm_*t2_ + 1/(1+qv2m)*cpme*self.t[k]*X) / cpmm*(1+qt) / (1+X);
                    t2md = (t2_ + X * self.t[k]) / (1+X);
                else:
                    cpme = cpd + cpv*self.q[k]
                    cpmm = cpd + cpv*qv2m + cpl*ql2m + cpi*qi2m
                    t2m  = (1/(1+qt_)*cpm_*t2_ + 1/(1+self.q[k])*cpme*self.t[k]*X) / cpmm*(1+qt) / (1+X)
                    t2md = (t2_ + X * self.t[k]) / (1+X)
    
                
                t2 = t2m;
                i = 0;
                not_converge = True;
                while not_converge:
                    i = i + 1;
                    #t = t2-1
                    if self.ice == 1:
                        fliq = np.max([np.min([(t2-self.newton_step_dt-self.tice)/(t0-self.tice), 1]), 0]);
                        fice = 1 - fliq;
                    else:
                        fliq = 1;
                        fice = 0;
                    
                    qv2 = np.min([qt, fliq*get_qvs(p2,t2-self.newton_step_dt) + fice*get_qvi(p2,t2-self.newton_step_dt)]);
                    qi2 = np.max([fice*(qt-qv2), 0]);
                    ql2 = np.max([qt-qv2-qi2, 0]);
                    cpm = cpd + cpv*qv2 + cpl*ql2 + cpi*qi2;
                    lhv = lv1 - lv2 * t2m;
                    lhs = ls1 - ls2 * t2m;
                    # lhv = lv1 - lv2 * (t2-1);
                    # lhs = ls1 - ls2 * (t2-1);
                    # fs2 = 1/(1+qt)*cpmm * (t2-1 - t2m) + ...
                    # lhv * (qv2-qv2m) - (lhs-lhv) * (qi2-qi2m);
                    fs2 = 1/(1+qt) * cpm * (t2 - self.newton_step_dt - t2m ) + \
                          lhv * (qv2-qv2m) - (lhs-lhv) * (qi2-qi2m)
                    
                    if self.ice == 1:
                        fliq = np.max([np.min([(t2-self.tice)/(t0-self.tice), 1]), 0]);
                        fice = 1 - fliq;
                    else:
                        fliq = 1;
                        fice = 0;
                    
                    qv2 = np.min([qt, fliq*get_qvs(p2,t2) + fice*get_qvi(p2,t2)]);
                    qi2 = np.max([fice*(qt-qv2), 0]);
                    ql2 = np.max([qt-qv2-qi2, 0]);
                    cpm = cpd + cpv*qv2 + cpl*ql2 + cpi*qi2;
                    #             lhv = lv1 - lv2 * t2;
                    #             lhs = ls1 - ls2 * t2;
                    #             fs1 = 1/(1+qt)*cpmm * (t2 - t2m) + ...
                    #                 lhv * (qv2-qv2m) - (lhs-lhv) * (qi2-qi2m);
                    fs1 = 1/(1+qt)*cpm  * (t2 - t2m) + \
                        lhv * (qv2-qv2m) - (lhs-lhv) * (qi2-qi2m)
                    dt2 = self.newton_step_dt * fs1 / (fs2-fs1)
                    
                    if i > 90:
                        print([i, t2, fs1, fs2, dt2])
                    
                    if i > 100:
                        print('lack of convergence, stopping iteration.')
                        break
                    
                    if np.abs(dt2) > converge:
                        t2 = t2 + dt2;
                    else:
                        not_converge = False
                        tv2 = t2 * (1 + reps * qv2) / (1 + qt) 
                        b2  = g * (tv2 - self.tv[k])/self.tv[k]    
                        self.dbpdze0[k,0] = g * (t2md - t2_) / self.t[k]    # dry mix
                        self.dbpdze0[k,1] = g * (t2m - t2md) / self.t[k]    # water heat exchange
                        self.dbpdze0[k,2] = - g / self.t[k] * lhv * (qv2-qv2m) / cpm*(1+qt)        # condensation
                        self.dbpdze0[k,3] =   g / self.t[k] * (lhs-lhv) * (qi2-qi2m) / cpm*(1+qt)  # fusion
                        self.dbpdze0[k,4] =   g * (reps - 1) * (qv2m - qv2_)
                        self.dbpdze0[k,5] = - g * (ql2m - ql2_)
                        self.dbpdze0[k,6] = - g * (qi2m - qi2_)
                        self.dbpdze0[k,7] =   g * (reps - 1) * (qv2 - qv2m)
                        self.dbpdze0[k,8] = - g * (ql2 - ql2m)
                        self.dbpdze0[k,9]= - g * (qi2 - qi2m)
                        self.dbpdze0[k,:] =  self.dbpdze0[k,:] / dz
            else:
                self.dbpdze0[k,:] = 0;
        #==============================================================================
        # Precipitation process, change mp, qp (condensate fall out, no latent heat)
            ql2_ = ql2
            qi2_ = qi2
            # determine how many condensate fall out of parcel
            if self.adiabat > 0:        
                if self.adiabat == 1 :        #all condensate fall out               
                    ql2 = 0; 
                    qi2 = 0;
                elif self.adiabat == 2:     # max mixing ratio specified
                    f   = np.min([ql2+qi2, self.qc_max])/(ql2+qi2);
                    ql2 = f * ql2;
                    qi2 = f * qi2;
                elif self.adiabat == 3:     # specific proportion
                    ql2 = ql2 * self.prc_w * dz;
                    qi2 = qi2 * self.prc_w * dz;
                else:
                    sys.stderr.write('Undefined adiabat')
                    raise SystemExit(1)
        #        m2  = (1 + qv2 + ql2 + qi2) / (1 + qt) * m2; % lose mass
                #qt is sum of mixing ratios calculated
                qt  = qv2 + ql2 + qi2;
                tv2 = t2 * (1 + reps*qv2) / (1 + qt)
                b2  = g  * (tv2 - self.tv[k])  / self.tv[k]
                #print(tv2,self.tv[k])
            
            self.dbpdzp0[k,0] = - g * (ql2 - ql2_);
            self.dbpdzp0[k,1] = - g * (qi2 - qi2_);
            self.dbpdzp0[k,:] = self.dbpdzp0[k,:] / dz;
        # store final buoyancy and related variables
            self.bp0[k]    = b2;
            self.tp0[k]    = t2;
            self.qp0[k, :] = [qv2, ql2, qi2];  
        # stop if b < 0 and p < 100 hPa
            if  (self.p[k] <= 10000) & (b2 < 0): 
                doit = False
            
        # save output results
        self.bp[self.ind]             = self.bp0         # parcel buoyancy
        self.tp[self.ind]             = self.tp0         # parcel temperature
        self.qp[self.ind]             = self.qp0 * 1000  # mixing ratio of vapor,liquid,ice
        self.other_dbpdzs[self.ind,:] = self.dbpdzs0
        self.other_dbpdze[self.ind,:] = self.dbpdze0
        self.other_dbpdzp[self.ind,:] = self.dbpdzp0
        self.td_in[self.ind]          = np.reshape(self.td,(-1,1)) 
        self.X[self.ind] = X_list
        #print(self.t.shape,self.bp0 .shape)
        self.td_in = np.squeeze(self.td_in)
        self.bp    = np.squeeze(self.bp)
        self.tp    = np.squeeze(self.tp)
        self.qp    = np.squeeze(self.qp)
        
        if self.z_opt == 1:
            self.other_zc[self.ind] = np.reshape(self.z,(len(self.z),-1))
   
    def plot_t(self):
        from metpy.plots import SkewT
#        import matplotlib.pyplot as plt
        from matplotlib.ticker import MultipleLocator, FormatStrFormatter

        #def draw_skewT_buoyancy(p_in,T_in,Td_in,Tp_in=[],bp=[],index=0):        
        p  = (self.p/100) * units('hPa')
        T  = (self.t * units('K')).to(units('degC'))
        Td = (self.td* units('K')).to(units('degC'))
        Tp = (np.squeeze(self.tp[self.ind])   * units('K')).to(units('degC'))
        bp = self.bp[self.ind]
        return(bp)
        #print(Tp.shape,T_in.shape)

#         fig  = plt.figure(figsize=(15,9))
#         skew = SkewT(fig,rotation=45,subplot=(1,2,1))
        
#         # Plot the data using normal plotting functions, in this case using
#         # log scaling in Y, as dictated by the typical meteorological plot
#         skew.plot(p, T, 'r',label='Temperature')
#         skew.plot(p, Td, 'g',label='Dew_T')
# #        style_list=['-', '--', '-.', ':',]
# #        label_list=['NoEntrain','ConstEntrain','DIA'] 
#         skew.plot(p, Tp,'k',linestyle=':',label='Parcel_T')
#         #skew.plot_barbs(p, u, v)
#         skew.ax.set_ylim(1000, 100)
#         skew.ax.set_xlim(-40, 60)
#         #print(p[0], T[0], Td[0])
#         # Calculate LCL height and plot as black dot
#         lcl_pressure, lcl_temperature = lcl(p[0], T[0], Td[0])
#         skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')
#         print('LCL:',lcl_pressure, lcl_temperature)
#         # Calculate full parcel profile and add to plot as black line
#         prof = parcel_profile(p, T[0], Td[0]).to('degC')
#         skew.plot(p, prof, 'k', linewidth=2,label='Ideal_Pacel_T')
#        plt.legend()
        
        # Shade areas of CAPE and CIN
#        skew.shade_cin(p, T, prof)
#        skew.shade_cape(p, T, prof)
#         skew.shade_cin(p, T, Tp)
#         skew.shade_cape(p, T, Tp)
        
#         # An example of a slanted line at constant T -- in this case the 0
#         # isotherm
#         skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)
        
#         # Add the relevant special lines
#         skew.plot_dry_adiabats()
#         skew.plot_moist_adiabats()
#         skew.plot_mixing_lines()
#         plt.legend() 
        
#         ax=fig.add_subplot(1,2,2)
#         ymajorLocator = MultipleLocator(1) #将y轴主刻度标签设置为0.5的倍数
#         ymajorFormatter = FormatStrFormatter('%d')
#         ax.yaxis.set_major_locator(ymajorLocator)
#         ax.yaxis.set_major_formatter(ymajorFormatter)
    
#         plt.grid(linestyle=':',linewidth=2)
        
#         #plotting final buoyancy below
#         plt.plot(bp,p,'k')
#         ax.yaxis.grid(True,which='major')    
#         plt.ylim([100,1000])
#        # ax.set_yticks(visible=False)
#         ax.set_yscale('log')
#         ax.invert_yaxis()
#         plt.xlabel("Buoyancy(0.01m/${s^2}$)")    
#         # Show the plot
#         plt.show()


# In[14]:


#=============================================================================
from metpy.calc import (find_intersections,
                        parcel_profile_with_lcl,#lfc,
                        cape_cin,lcl,el,
                        )#, 
import metpy.constants as mpconsts
from metpy.units import concatenate,units

class data_struct():
    def __init__(self):
        self.p = []
        self.t = []
        self.dp = []
        self.ept = []
def _find_append_zero_crossings(x, y):
    r"""
    Find and interpolate zero crossings.

    Estimate the zero crossings of an x,y series and add estimated crossings to series,
    returning a sorted array with no duplicate values.

    Parameters
    ----------
    x : `pint.Quantity`
        x values of data
    y : `pint.Quantity`
        y values of data

    Returns
    -------
    x : `pint.Quantity`
        x values of data
    y : `pint.Quantity`
        y values of data

    """
    # Find and append crossings to the data
    crossings = find_intersections(x[1:], y[1:], np.zeros_like(y[1:]) * y.units)
    x = concatenate((x, crossings[0]))
    y = concatenate((y, crossings[1]))

    # Resort so that data are in order
    sort_idx = np.argsort(x)
    x = x[sort_idx]
    y = y[sort_idx]

    # Remove duplicate data points if there are any
    keep_idx = np.ediff1d(x, to_end=[1]) > 0
    x = x[keep_idx]
    y = y[keep_idx]
    return x, y

def _greater_or_close(a, value, **kwargs):
    r"""Compare values for greater or close to boolean masks.
    """
    return (a > value) | np.isclose(a, value, **kwargs)


def _less_or_close(a, value, **kwargs):
    r"""Compare values for less or close to boolean masks.
    """
    return (a < value) | np.isclose(a, value, **kwargs)
    


# In[9]:


class sounding_cal():
    
    def __init__(self,pressure, temperature, dewpt, 
                 parcel_temperature_profile=None,
                 specific_pressure_start=None,
                 specific_pressure_end=None):
        self.pressure    = pressure
        self.temperature = temperature
        self.dewpt       = dewpt
        self.parcel_temperature_profile = parcel_temperature_profile  
        self.specific_pressure_start = specific_pressure_start #level for integrate,unit:Pa
        self.specific_pressure_end   = specific_pressure_end   #level for integrate
        
        if parcel_temperature_profile is None:
            
            new_stuff = parcel_profile_with_lcl(pressure, temperature, dewpt)
            
            #annie code
            pressure, temperature, dewpt, parcel_temperature_profile = new_stuff
            
#             pressure, temperature, _, parcel_temperature_profile = new_stuff
            
            temperature = temperature.to('degC')
            
            self.parcel_temperature_profile = parcel_temperature_profile.to('degC')
        
        #annie code
        self.pressure = pressure
        self.dewpt = dewpt
        self.temperature = temperature
        
        self.lfc = data_struct()
        self.el  = data_struct()
        self.cal_lfc()
        self.cal_el()
        self.integrate_buoyancy()
        self.lfc.p=self.lfc.p.to(units('hPa')) 
        self.lfc.ept=self.lfc.ept.to(units('degK')) 
        self.el.p=self.el.p.to(units('hPa')) 
        self.el.ept=self.el.ept.to(units('degK'))  
    
      #self.info()
        
    def cal_lfc(self):
        
        x, y = find_intersections(self.pressure[1:], self.parcel_temperature_profile[1:], self.temperature[1:], direction='increasing')
        
#         print(x,y)

  #=======================LFC====================================================
      # The LFC could:
      # 1) Not exist
      # 2) Exist but be equal to the LCL
      # 3) Exist and be above the LCL
      #print(x,y)
      # LFC does not exist or is LCL
        if len(x) == 0:
            self.lfc.p = np.nan * self.pressure.units
            self.lfc.t = np.nan * self.temperature.units
            #            if np.all(_less_or_close(self.parcel_temperature_profile, self.temperature)):
            #                # LFC doesn't exist
            #                self.lfc.p = np.nan * self.pressure.units
            #                self.lfc.t = np.nan * self.temperature.units
            #            else:  # LFC = LCL
            #                self.lfc.p, self.lfc.t = lcl(self.pressure[0], self.temperature[0], self.dewpt[0])

            # LFC exists and is not LCL. Make sure it is above the LCL.
        else:
            idx = x < lcl(self.pressure[0], self.temperature[0], self.dewpt[0])[0]
            if sum(idx)>0:
                self.lfc.p = x[idx][0]
                self.lfc.t = y[idx][0]
            else:
                self.lfc.p = np.nan * self.pressure.units
                self.lfc.t = np.nan * self.temperature.units
        #print(lcl(self.pressure[0], self.temperature[0], self.dewpt[0])[0])
        if np.isnan(self.lfc.p):
            self.lfc.dp = np.nan* self.pressure.units
            self.lfc.ept = np.nan* self.temperature.units
        else:
            
            #annie code
            self.lfc.dp  = np.interp(self.lfc.p, self.pressure, self.dewpt)
#             self.lfc.dp  = np.interp(self.lfc.p.magnitude, self.pressure, self.dewpt)*units('degK')
            self.lfc.ept =  equivalent_potential_temperature(self.lfc.p,self.lfc.t,self.lfc.dp) 


    def cal_el(self):

        if self.parcel_temperature_profile[-1] > self.temperature[-1]:
            self.el.p = np.nan * self.pressure.units
            self.el.t = np.nan * self.temperature.units
        x, y = find_intersections(self.pressure[1:], self.parcel_temperature_profile[1:],
                              self.temperature[1:], direction='decreasing')
        # Otherwise the last intersection (as long as there is one) is the EL     
        if len(x) > 0:
            self.el.p = x[-1]
            self.el.t = y[-1]            
        else:
            self.el.p = np.nan * self.pressure.units
            self.el.t = np.nan * self.temperature.units

        if np.isnan(self.el.p):
            self.el.dp = np.nan* self.pressure.units
            self.el.ept = np.nan* self.temperature.units
        else:
            self.el.dp  = np.interp(self.el.p, self.pressure , self.dewpt)
#                 self.el.dp  = np.interp(self.el.p.magnitude, self.pressure , self.dewpt)*units('degK')
            self.el.ept =  equivalent_potential_temperature(self.el.p,self.el.t,self.el.dp)  

    def integrate_buoyancy(self):

        if self.specific_pressure_start == None:
            if ~np.isnan(self.lfc.p):
                self.s_start = self.lfc.p.magnitude
            else:
                self.s_start = 65000 
        else:
            self.s_start = self.specific_pressure_start.to(units('Pa')).magnitude

        if self.specific_pressure_end == None:
            #print(self.el.p.magnitude,self.s_start)
            if np.all(((~np.isnan(self.el.p)) , (self.el.p.magnitude < self.s_start))) :
                self.s_end = self.el.p.magnitude
            else:
                self.s_end = self.pressure[-1].magnitude
        else:
            self.s_end = self.specific_pressure_end.to(units('Pa')).magnitude       
        #        if self.s_end >= self.s_start:
        #            self.s_end = self.pressure[-1].magnitude     # if there is not el, integrate to 100hPa
        #        

        y = (self.parcel_temperature_profile - self.temperature).to(units.degK)

        
        x, y = _find_append_zero_crossings(np.copy(self.pressure), y)
        
        if (~np.isnan(self.lfc.p.magnitude)) & (~np.isnan(self.el.p.magnitude)):
          # positive energy---cape like
          # print(x,self.lfc.p)
            
            p_mask = _less_or_close(x, self.lfc.p) & _greater_or_close(x,  self.el.p)
#             p_mask = _less_or_close(x, self.lfc.p.magnitude) & _greater_or_close(x,  self.el.p.magnitude)
            x_clipped = x[p_mask]
            
            y_clipped = y[p_mask]
        
        
#             print("working here")
            
            x_clipped = x_clipped.magnitude
            y_clipped = y_clipped.magnitude
            
            self.p_buoyancy = (mpconsts.Rd * (np.trapz(y_clipped, np.log(x_clipped)) * units.degK)).to(units('J/kg'))
            
#             print(self.p_buoyancy)
            
            #negative energy---cin like
            
            p_mask = _greater_or_close(x, self.lfc.p)
            
#             p_mask = _greater_or_close(x, self.lfc.p.magnitude)
            x_clipped = x[p_mask]
            y_clipped = y[p_mask]
        
            x_clipped = x_clipped.magnitude
            y_clipped = y_clipped.magnitude
            
            #print(x_clipped)
            self.n_buoyancy  = (mpconsts.Rd
                 * (np.trapz(y_clipped, np.log(x_clipped)) * units.degK)).to(units('J/kg'))
        else:
            self.p_buoyancy = np.nan*units('J/kg')
            self.n_buoyancy = np.nan*units('J/kg')
            
        return self.p_buoyancy

    # # positive energy---cape like
    # p_mask = _less_or_close(x, self.s_start) & _greater_or_close(x,  self.s_end)
    # x_clipped = x[p_mask]
    # y_clipped = y[p_mask]
    # self.p_buoyancy = (mpconsts.Rd
    #     * (np.trapz(y_clipped, np.log(x_clipped)) * units.degK)).to(units('J/kg'))
    # #negative energy---cin like
    # p_mask = _greater_or_close(x, self.s_start)
    # x_clipped = x[p_mask]
    # y_clipped = y[p_mask]
    # #print(x_clipped)
    # self.n_buoyancy  = (mpconsts.Rd
    #        * (np.trapz(y_clipped, np.log(x_clipped)) * units.degK)).to(units('J/kg'))

#         x = self.pressure.magnitude
#         y = (self.parcel_temperature_profile - self.temperature).to(units.degK)
#         # energy above        
#         p_mask = _less_or_close(self.pressure.magnitude,self.s_start) & _greater_or_close(self.pressure.magnitude,self.s_end)
#         x_clipped = x[p_mask]
#         y_clipped = y[p_mask]
#         self.buoyancy_above = (mpconsts.Rd
#                * (np.trapz(y_clipped[::-1], np.log(x_clipped[::-1])) * units.degK)).to(units('J/kg'))

#         self.x = x_clipped
#         self.y=y_clipped
#         # energy below
#         p_mask = _greater_or_close(self.pressure.magnitude, self.s_start)
#         x_clipped = x[p_mask]
#         y_clipped = y[p_mask]
#         # print(y_clipped)
#         self.buoyancy_below  = (mpconsts.Rd
#                * (np.trapz(y_clipped[::-1], np.log(x_clipped[::-1])) * units.degK)).to(units('J/kg'))
    def info(self):
        print('INTEGRATION PROCESS:')    
        print("\tTop level:", self.s_end/100*units('hPa'))
        print("\tStart level:", self.s_start/100*units('hPa'))
        print('\tIntegrate Buoyancy Above:',self.buoyancy_above)
        print('\tIntegrate Buoyancy Below:',self.buoyancy_below)
        print('\tCAPE:',self.p_buoyancy)
        print('\tCIN:',self.n_buoyancy)

